# Homework Batch III: Routing Algorithms
## Davide Basso - SM3500450

### Exercise 1

Implement the binary heap-based version of the Dijkstra's algorithm.

First of all I've done some slight modifications on the binheap class that we've implemented during lesson in order to properly deal with nodes. These can be found in the file **node_binheap.py** present in this repository; basically since we are considering distances in order to build the heap, all the comparison steps were done with respect to distances and not node's index. Further insights can be found as comments in the very same file.

Also classes such as `Node`, `Edge` and `Weighted_Graph` have been implemented, within **graph.py**. Also in this case I recall comments present in the aforementioned file, in order to grasp all the properties and characteristics of each object.

Finally here below the implementation for Dijkstra's algorithm using bin-heaps as priority queues:

In [1]:
from math import inf # necessary to initialize distances of the graph
from graph import *
from node_binheap import binheap

In [2]:
def init_SSSP(G):
    for v in G.V:
        v.d = inf
        v.pred = None

def relax(Q, u, v, w):
    if u.d+w < v.d:
        Q.decrease_key(v.heap_index, u.d+w)
        v.pred = u

def Dijkstra(G, s):
    init_SSSP(G)
    s.d = 0

    Q = binheap(G.V)
    for h_i, node in enumerate(Q._A):
        node.heap_index = h_i

    final_queue = []
    i = 0
    while not Q.is_empty():
        u = Q.remove_minimum()
        final_queue.append(u)

        # some prints in order to check if we are following the correct path
        if i != 0:
            print(f"\nnode {u.index} with total distance {u.d} from root and predecessor node is {u.pred.index}")
        else:
            i += 1
            print(f"The path starts from node {u.index}, then we have:")

        for (v,w) in G.Adj[u.index]:
            relax(Q, u, v, w)
    
    return final_queue

We can now test if the algorithm is able to find the shortest paths from a given source to all the other nodes belonging to the graph (this example refers to the one present in the slides of the course, more precisely at ch.7 slide n°13).

In [3]:
a = Node(0,None,None,None)
b = Node(1,None,None,None)
c = Node(2,None,None,None)
d = Node(3,None,None,None)
e = Node(4,None,None,None)
f = Node(5,None,None,None)

vertices = [a,b,c,d,e,f]
edges = [Edge(a,b,1), Edge(a,c,5), Edge(b,f,15), Edge(c,d,2), Edge(d,e,1), Edge(e,f,3)]

In [4]:
G = Weighted_Graph(vertices, edges)
res = Dijkstra(G,a)

The path starts from node 0, then we have:

node 1 with total distance 1 from root and predecessor node is 0

node 2 with total distance 5 from root and predecessor node is 0

node 3 with total distance 7 from root and predecessor node is 2

node 4 with total distance 8 from root and predecessor node is 3

node 5 with total distance 11 from root and predecessor node is 4


Final result is correct and coherent with the one shown in slides.

### Exercise 2

Consider the contraction hierarchies presented during the course. Assume to deal with graphs that can be fully represented in the memory of your computer. Implement:

(a) an algorithm to add the shortcuts to a graph;

Shortcuts are pretty useful in the case we'd like to compute shortest paths in contraction hierarchies or more general in some kind of graph by skipping all those nodes that are less important with respect to the last reached one. A shortcut is added to the graph if and only if the path obtained by merging incoming and outcoming edges is smaller than other edges already present and connecting neighbours of skipped node.

To do so I've implemented a function that takes as inputs the node we want to skip, all the edges and vertices present in the graph while as output it returns the updated list of edges with all the shortcuts needed.

In [5]:
def compute_shortcut(node, edges, vertices, contracted):
    shortcuts = {"deleted": None,
                 "pred": [], "pred_weight":[],
                 "next": [], "next_weight": []}

    shortcuts["deleted"] = node.index

    # find the edges in which node is involved 
    # and add them to the dictionary
    for edge in edges:
        if edge.v.index == node.index:
            shortcuts["pred"].append(edge.u.index)
            shortcuts["pred_weight"].append(edge.weight)
        if edge.u.index == node.index:
            shortcuts["next"].append(edge.v.index)
            shortcuts["next_weight"].append(edge.weight)    

    for k,v in shortcuts.items():
        print(f"{k} -> {v}")
    
    # remove edges involving node we want to skip
    res = [edge for edge in edges if not (edge.v.index == node.index or edge.u.index == node.index)]
    
    # create shortcuts
    new_edges = []
    for pred, pred_w in zip(shortcuts["pred"], shortcuts["pred_weight"]):
        p = [v for v in vertices if v.index == pred][0]
        for succ, succ_w in zip(shortcuts["next"], shortcuts["next_weight"]):
            s = [v for v in vertices if v.index == succ][0]
            new_edge = Edge(p, s, pred_w+succ_w)
            new_edges.append(new_edge)
            res.append(new_edge)
    
    # compare if new edges are worse than already existing ones
    # if so remove them
    for n_e in new_edges:
        for e in edges:
            if e.u.index == n_e.u.index and e.v.index == n_e.v.index and e.weight < n_e.weight:
                res.remove(n_e)
            else:
                # We add in the contracted matrix the middle vertex that is skipped due to the shortcut; 
                # it is placed in [n,m] position where:
                # • n is shortcut source's index 
                # • m is shortcut destination's index
                contracted[n_e.u.index][n_e.v.index] = node
                
    return res

Let's test the function on the previous example by removing node c:

In [6]:
contr = [[None for v in vertices] for j in vertices]
res = compute_shortcut(c, edges, vertices, contr)

print("\nNew edges with shortcut:")
for el in res:
    print(f"pred: {el.u.index}, succ: {el.v.index}, val: {el.weight}")

deleted -> 2
pred -> [0]
pred_weight -> [5]
next -> [3]
next_weight -> [2]

New edges with shortcut:
pred: 0, succ: 1, val: 1
pred: 1, succ: 5, val: 15
pred: 3, succ: 4, val: 1
pred: 4, succ: 5, val: 3
pred: 0, succ: 3, val: 7


And we can observe that even in this case the algorithm gives the correct result (both in terms of skipped node and new edges' value).

(b) a bidirectional version of Dijkstra algorithm that can operate on the graphs decorated by the algorithm at Point 2a.

From theory, in order to correctly implement a bidirectional version of Dijkstra, we know that we need to operate on a forward and reverse graph (a graph in which all edges are flipped). 
To achieve this first step I've implemented the `reverse_edges` function, that simply returns a list of edges with source and destination inverted.

In [7]:
def reverse_edges(edges):
    back = []
    for el in edges:
        tmp = Edge(el.v, el.u, el.weight)
        back.append(tmp)

    return back

Dijkstra bidirectional algorithm is based on the following ideas:
* Start simultaneously Dijkstra algorithm from source and target node respectively in forward and backward graph;
* when the two executions finalize at the same time the same node or when the finalized node of one execution has been already processed in the other one then both executions are stopped and forward and backward path are retrieved (using `get_predecessor` function);
* if shortcuts have been added then, before relaxing an edge, we need to check if destination is leading to a node with higher importance in the hierarchy w.r.t. the source one (only in this way we can exploit the advantage of contraction hierarchies and shortcuts).


In [8]:
def get_predecessors(node, predecs):
    if node.pred is not None:
        predecs.append(node.pred)
        get_predecessors(node.pred, predecs)

In [42]:
from copy import deepcopy
def bidirect_Dijkstra(f_G, b_G, s, t, shortcuts=False):
    # Deep copy is needed, otherwise when updating 
    # the value of a node that is propagated also 
    # to correspondent node belonging to the other graph
    F_G = deepcopy(f_G)
    B_G = deepcopy(b_G)
    init_SSSP(F_G)
    init_SSSP(B_G)

    F_G.V[s.index].d = 0
    f_Q = binheap(F_G.V)

    B_G.V[t.index].d = 0
    b_Q = binheap(B_G.V)

    for h_i, node in enumerate(f_Q._A):
        node.heap_index = h_i
    for h_i, node in enumerate(b_Q._A):
        node.heap_index = h_i

    forward_processed = []
    backward_processed = []
    i = 0
    final_weight = 0
    while not f_Q.is_empty() and not b_Q.is_empty():
        u = f_Q.remove_minimum()
        v = b_Q.remove_minimum()

        flag = False

        f_queue = []
        b_queue = []
        dist = inf

        if u.index==v.index:
            flag = True
            f_queue.append(u)
            get_predecessors(u,f_queue)
            b_queue.append(v)
            get_predecessors(v,b_queue)

            f_queue.reverse()
            final_weight = f_queue[-1].d + b_queue[0].d
            f_queue.pop()
            break
        
        else:
            if u.index in backward_processed:
                b_node = [x for x in B_G.V if x.index == u.index][0]
                if u.d + b_node.d < dist:
                    flag = True
                    get_predecessors(u,f_queue)
                    b_queue.append(b_node)
                    get_predecessors(b_node,b_queue)
                    dist = u.d + b_node.d

            elif v.index in forward_processed:
                f_node = [x for x in F_G.V if x.index == u.index][0]
                if v.d + f_node.d < dist:
                    flag = True
                    b_queue.append(v)
                    get_predecessors(v,b_queue)
                    get_predecessors(f_node,f_queue)
                    dist = v.d + f_node.d
            
            if flag is True:
                f_queue.reverse()
                final_weight = dist
                break
        
        forward_processed.append(u.index)
        backward_processed.append(v.index)
        
        if i != 0:
            print(f"\nforward node {u.index} with total distance {u.d} from root and predecessor node is {u.pred.index}")
            print(f"backward node {v.index} with total distance {v.d} from root and predecessor node is {v.pred.index}")
        else:
            i += 1
            print(f"The forward path starts from node {u.index}, then we have:")
            print(f"The backward path starts from node {v.index}, then we have:")
        

        for (k,w) in F_G.Adj[u.index]:
            if shortcuts is True:
                if u.index <= k.index:
                    relax(f_Q, u, k, w)
            else:
                relax(f_Q, u, k, w)
            
        for (k,w) in B_G.Adj[v.index]:
            if shortcuts is True:
                if v.index <= k.index:
                    relax(b_Q, v, k, w)
            else:
                relax(b_Q, v, k, w)
    
    final_path = f_queue+b_queue
        
    return final_path, final_weight


In [47]:
def contraction(vertices, edges, updated_edges, num_og_edges, contracted, f=compute_shortcut, index=0):
    '''
    Function used to compute all the shortcuts for edges in the graph
    that have as destination a less important node w.r.t source one
    '''
    if index == num_og_edges:
        return updated_edges

    edge = edges[index]
    index += 1
    if edge.v.index < edge.u.index:
        new = f(edge.v, updated_edges, vertices, contracted)
        return contraction(vertices, edges, new, num_og_edges, contracted, f, index)
    else:
        return contraction(vertices, edges, updated_edges, num_og_edges, contracted, f, index)

Let's test it first on a graph without any kind of shortcuts. I've taken nodes' indices as importance metric, and this will be valid for all the next examples too.

In [58]:
a = Node(0,None,None,None)
b = Node(1,None,None,None)
c = Node(2,None,None,None)
d = Node(5,None,None,None)
e = Node(4,None,None,None)
f = Node(3,None,None,None)

vertices = [a,b,c,d,e,f]
edges = [Edge(a,b,1), Edge(a,c,10), Edge(b,c,1), Edge(b,f,15), Edge(c,d,2), Edge(d,e,1), Edge(e,f,3)]
backward_edges = reverse_edges(edges)

G1 = Weighted_Graph(vertices, edges)
G2 = Weighted_Graph(vertices, backward_edges)

In [59]:
res, weight = bidirect_Dijkstra(G1, G2, a, e, shortcuts=False)

The forward path starts from node 0, then we have:
The backward path starts from node 4, then we have:

forward node 1 with total distance 1 from root and predecessor node is 0
backward node 5 with total distance 1 from root and predecessor node is 4


In [60]:
for el in res:
    el.print_node()
print(f"Resulting distance from node a to e is: {weight}")

index: 0, data: 0, predecessor: None
index: 1, data: 1, predecessor: 0
index: 2, data: 3, predecessor: 5
index: 5, data: 1, predecessor: 4
index: 4, data: 0, predecessor: None
Resulting distance from node a to e is: 5


And result is correct.

We can now test the algorithm on the graph presented at ch.8 slide n°45. All shortcuts are created and added to original edges in order to have a top view of the hierarchy.

In [51]:
a = Node(0,None,None,None)
b = Node(1,None,None,None)
c = Node(2,None,None,None)
d = Node(3,None,None,None)
e = Node(4,None,None,None)
f = Node(5,None,None,None)
g = Node(6,None,None,None)
h = Node(7,None,None,None)

vertices = [a,b,c,d,e,f,g,h]
edges = [Edge(b,c,2), Edge(c,b,1), Edge(c,d,3), Edge(a,e,1), Edge(e,a,3), Edge(h,a,1), Edge(h,g,1), Edge(g,h,1), Edge(d,h,3), Edge(e,f,1), Edge(f,h,1), Edge(a,f,1)]

contr = [[None for v in vertices] for j in vertices]

forward_edges = contraction(vertices, edges, contracted=contr, updated_edges=edges, num_og_edges=len(edges))
edges.extend(forward_edges)

G1 = Weighted_Graph(vertices, edges)
G2 = Weighted_Graph(vertices, reverse_edges(edges))

deleted -> 1
pred -> [2]
pred_weight -> [1]
next -> [2]
next_weight -> [2]
deleted -> 0
pred -> [4, 7]
pred_weight -> [3, 1]
next -> [4, 5]
next_weight -> [1, 1]
deleted -> 0
pred -> []
pred_weight -> []
next -> []
next_weight -> []
deleted -> 6
pred -> [7]
pred_weight -> [1]
next -> [7]
next_weight -> [1]


In [52]:
res, weight = bidirect_Dijkstra(G1, G2, c, e, shortcuts = True)
print("\nShortest path form node c to e using shortcuts:")
for el in res:
    el.print_node()
print(f"Resulting distance from node c to e is: {weight}")

The forward path starts from node 2, then we have:
The backward path starts from node 4, then we have:

forward node 3 with total distance 3 from root and predecessor node is 2
backward node 7 with total distance 2 from root and predecessor node is 4

Shortest path form node c to e using shortcuts:
index: 2, data: 0, predecessor: None
index: 3, data: 3, predecessor: 2
index: 7, data: 2, predecessor: 4
index: 4, data: 0, predecessor: None
Resulting distance from node c to e is: 8


Result also in this case seems to be correct, since node 0 is not considered due to the presence of a shortcut, but the output path still needs to be unpacked.
In order to do so we can recursively unpack the route by using the *contracted* matrix.

In [53]:
def shortcuts_unpacking(path, contracted, index):
    to_be_added = contracted[path[index-1].index][path[index].index]
    if to_be_added is None:
        return

    else:
        path.insert(index, to_be_added)
        shortcuts_unpacking(path, contracted, index)
        return

In [54]:
def path_unpacking(path, contracted):
    for i in range(1,len(path)):
        if contracted[path[i-1].index][path[i].index] is not None:
            shortcuts_unpacking(path, contracted, i)
        else:
            continue

In [55]:
path_unpacking(res, contr)

In [56]:
i=0
print("This is the overall shortest path obtained by including also contracted nodes:\n")
for el in res:
    if i != len(res)-1:
        i += 1
        print(f"{el.index} -> ", end='')
    else:
        print(f"{el.index}")


This is the overall shortest path obtained by including also contracted nodes:

2 -> 3 -> 7 -> 0 -> 4


Finally we can verify that previous result is correct by comparing it to the one obtained by computing the same path but without any shortcuts.

In [20]:
a = Node(0,None,None,None)
b = Node(1,None,None,None)
c = Node(2,None,None,None)
d = Node(3,None,None,None)
e = Node(4,None,None,None)
f = Node(5,None,None,None)
g = Node(6,None,None,None)
h = Node(7,None,None,None)

vertices = [a,b,c,d,e,f,g,h]
edges = [Edge(b,c,2), Edge(c,b,1), Edge(c,d,3), Edge(a,e,1), Edge(e,a,3), Edge(h,a,1), Edge(h,g,1), Edge(g,h,1), Edge(d,h,3), Edge(e,f,1), Edge(f,h,1), Edge(a,f,1)]
backward_edges = reverse_edges(edges)
G1 = Weighted_Graph(vertices, edges)
G2 = Weighted_Graph(vertices, backward_edges)

res, weight = bidirect_Dijkstra(G1,G2,c,e, shortcuts=False)

The forward path starts from node 2, then we have:
The backward path starts from node 4, then we have:

forward node 1 with total distance 1 from root and predecessor node is 2
backward node 0 with total distance 1 from root and predecessor node is 4

forward node 3 with total distance 3 from root and predecessor node is 2
backward node 7 with total distance 2 from root and predecessor node is 0


In [21]:
print("Shortest path form node c to e without using shortcuts:")
for el in res:
    el.print_node()
print(f"Resulting distance from node c to e is: {weight}")

Shortest path form node c to e without using shortcuts:
index: 2, data: 0, predecessor: None
index: 3, data: 3, predecessor: 2
index: 7, data: 2, predecessor: 0
index: 0, data: 1, predecessor: 4
index: 4, data: 0, predecessor: None
Resulting distance from node c to e is: 8
